In [ ]:
# CUDA_VISIBLE_DEVICES=0,1,2,3 vllm serve LGAI-EXAONE/EXAONE-4.0-32B --port 8005 --served-model-name exaone-4-32b --data-parallel-size 4
#? 위와 같은 명령어로 GPU 하나에다가 EXAONE 실행함

In [1]:
import mlflow
import dspy
import glob
import os
import pandas as pd
from dspy.evaluate import Evaluate

In [2]:
DATA_DIRECTORY = "/home/eoeldroal/WorkPlace/Legal/Legal-DSPy/Data/LU"

In [30]:
mlflow.set_tracking_uri("http://127.0.0.1:5001")
mlflow.set_experiment("DSPy")
mlflow.dspy.autolog()

### 문제 1 -> think 활성화 관련 문제.
- 원인 : LiteLLM을 응용한 openai sdk에서, enable_thinking을 하지 않음
- 해결 : dspy.LM의 extra_body 인자를 통해서 해결함.

In [120]:
lm = dspy.LM(
    'openai/exaone-4-32b', 
    api_key='no', 
    api_base='http://0.0.0.0:8005/v1',
    # thinking 모드 활성화를 위한 extra_body 설정
    extra_body={"chat_template_kwargs": {"enable_thinking": True}},
    # EXAONE-4 thinking 모드에 권장되는 파라미터들
    temperature=0.6,
    top_p=0.95,
    max_tokens=16384,
) 
# 이렇게 뒷부분에 v1를 명시해줘야 올바른 엔드포인트로 간다.
# 권장되는 설정인 temp와 top p 도 설정하고,
# 기존에는 think를 하지 않도록 설정되어 있는 것을 이제 extra_body 인자를 통해서 항상 강제하자. 
o4_mini = dspy.LM('openai/o4-mini', max_tokens=20_000, temperature = 1.0)
dspy.configure(lm = lm, track_usage=True)

In [4]:
lm("CoT 방법에 대해서 단계적으로 자세히 설명해 주세요.")
# 아래와 같이 전달됨.
# [|user|]\nCoT 방법에 대해서 단계적으로 자세히 설명해 주세요.[|endofturn|]\n[|assistant|]\n<think>\n

['Hmm, the user is asking for a detailed step-by-step explanation of the CoT (Chain of Thought) method. This seems like a technical question about AI reasoning techniques. \n\nI recall that CoT is a popular prompt engineering strategy in large language models. The user might be a researcher, developer, or student working with AI systems. They probably need this for practical application rather than just curiosity, given how specific the request is.\n\nThe query is in Korean but straightforward. No emotional cues detected - this appears to be a pure knowledge-seeking request. I should maintain technical accuracy while keeping it accessible. \n\nBreaking this down: First I need to define CoT clearly, then explain why it\'s needed (the reasoning gap in LLMs), followed by the actual step-by-step mechanics. The examples are crucial - abstract explanations won\'t help as much as concrete cases. \n\nI\'m considering including both the standard implementation and recent variations like self-co

- 결과를 보면, `<think>` 뒷부분을 완성하는 식으로 구성되었다는 것을 알 수 있다. 
- `</think>` 가 이제 reasoning과 answer를 가리는 중요한 부분이라는 사실도 알 수 있다. 
- 이외의 special token은 모두 정리된 것 같다.
- 이렇게 LM에 직접적으로 접근할 경우, 시그니처나 모듈을 사용할 때와는 달리 거의 raw에 가까운 응답을 제시하는 것 같다. 
    - 즉, 시그니처나 모듈을 사용할 떄는 mlflow를 통한 검토가 필수일 것이다.

In [12]:
# 아래처럼 구성할 경우, 어떻게 될까? 
COT = dspy.ChainOfThought("request -> answer")

In [14]:
# 일단, 시그니처에 없던, 응답 부분에 reasoning이 알아서 추가되었다는 사실을 알 수 있다. 
COT

predict = Predict(StringSignature(request -> reasoning, answer
    instructions='Given the fields `request`, produce the fields `answer`.'
    request = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Request:', 'desc': '${request}'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
))

In [ ]:
with mlflow.start_run(run_name="cot_single_test"):
    test_result = COT(**{"request" : "CoT 방법에 대해서 단계적으로 자세히 설명해 주세요."})
    # 이렇게 **를 사용해야 한다. 그래야 이러한 request 라고 시그니처에 지정된 입력 인자와 대응이 되는 것을 알 수 있다.

🏃 View run cot_single_test at: http://127.0.0.1:5001/#/experiments/366534923737239846/runs/b5fdab35073a4bff80ba9549148e996c
🧪 View experiment at: http://127.0.0.1:5001/#/experiments/366534923737239846


Trace(trace_id=tr-7d34942c7875a3b290031fafb546f8a7)

In [19]:
test_result

Prediction(
    reasoning='The user has requested a detailed step-by-step explanation of the Chain-of-Thought (CoT) method. \nCoT is a technique used in problem-solving and reasoning, especially in artificial intelligence and cognitive tasks, where the solution is broken down into a sequence of intermediate steps. \nThe explanation should cover:\n- Definition and purpose of CoT.\n- The step-by-step process of how CoT is applied.\n- The benefits of using CoT.\n- An example to make it concrete.\n\nWe will write the answer in Korean as the request is in Korean.',
    answer='Chain-of-Thought (CoT) 방법은 문제 해결이나 추론 과정에서 최종 답에 이르기까지의 사고 과정을 단계별로 나누어 설명하는 기법입니다. 이 방법은 복잡한 문제를 작은 단계로 분해하여 체계적으로 접근할 수 있도록 도와줍니다. CoT는 특히 인공지능 모델에서 복잡한 질문에 대한 답변을 생성할 때 유용하게 사용됩니다.\n\nCoT 방법의 단계별 설명:\n\n1. **문제 이해 단계**: 주어진 문제를 정확히 이해하고, 무엇을 요구하는지 파악합니다. 문제의 입력과 기대되는 출력을 명확히 합니다.\n\n2. **중간 단계 설계**: 문제를 해결하기 위해 필요한 중간 단계들을 계획합니다. 각 단계는 이전 단계의 결과를 바탕으로 자연스럽게 연결되어야 합니다.\n\n3. **단계별 실행**: 각 중간 단계를 차례대로 실행하며, 각 단계에서 필요

mlflow를 통해서 raw 출력을 관찰해 보니, 아래와 같은 결과가 나왔다. 
```
We are given a request: "CoT 방법에 대해서 단계적으로 자세히 설명해 주세요." We need to explain the Chain-of-Thought (CoT) ...
Let's structure the answer accordingly.
[[ ## reasoning ## ]] The user has requested a detailed step-by-step explanation of the Chain-of-Thought (CoT) method. CoT is a technique used in problem-solving and reasoning, especially in artificial intelligence and cognitive tasks, ...
The benefits of using CoT.
An example to make it concrete.
We will write the answer in Korean as the request is in Korean.
[[ ## answer ## ]] Chain-of-Thought (CoT) 방법은 문제 해결이나 추론 과정에서 최종 답에 이르기까지의 사고 과정을 단계별로 나누어 설명하는 기법입니다. 이 방법은 복잡한 문제를 작은 단계로 분해하여 체계적으로 접근할 수 있도록 도와줍니다. CoT는 특히 인공지능 모델에서 복잡한 질문에 대한 ...
따라서 남자의 비율은 60%입니다.
이처럼 CoT는 문제를 단계별로 나누어 체계적으로 해결하는 방법입니다.
[[ ## completed ## ]] </think>

[[ ## reasoning ## ]] 사용자가 CoT(Chain-of-Thought) 방법에 대한 단계별 상세 설명을 요청했습니다. CoT는 복잡한 문제 해결 시 사고 ...
실제 사례를 통한 작동 원리 시연
주요 장점 및 활용 분야 강조
[[ ## answer ## ]] CoT(Chain-of-Thought) 방법은 복잡한 문제를 해결할 때 사고의 연쇄적 흐름을 체계적으로 분해하는 기법입니다. 인공지능, 논리 추론, 문제 해결 분야에서 널리 활용되며, 단계별 설명은 다음과 같습니다:
...
인공지능: 복잡한 질의응답(QA) 시스템
소프트웨어 테스팅: 알고리즘 디버깅
이처럼 CoT는 "문제 → 하위 작업 → 계산 → 통합" 의 단계적 프레임워크로, 체계적인 사고를 가능하게 합니다.
```
이러한 식으로, 형식이 뒤죽박죽이 된다.
모델이 준수해야 하는 형식은 2가지이다.
- 기존 EXAONE 모델의 `<think>`, `</think>`와 같은 스페셜 토큰을 응용한 형식
- DSPy 모듈과 시그니처로 인한 `[[ ## reasoning ## ]]`, `[[ ## answer ## ]]`과 같은 부분
이 두 부분이 섞여 있고, 중복되어 나타난다.
- 또한, `[[ ## completed ## ]]` 토큰은 이제 마지막 끝을 나타내는 마커인데, 이를 `think`가 끝날 때 사용하여서 이제 `ChatAdapter.parse` 로 하여금 최종 응답 부분을 제대로 파싱하도록 하지 않고, `think` 내부에 제시된 부분을 제시하도록 하여서 의도와 다르게 흘러간다.
- 즉, 전반적으로 모델은 이미 스페셜 토큰을 통해 `think`를 하도록 구성되어 있는데, 모듈과 시그니처로 인해 중복된 reasoning 요청이 들어갔고, 또한 애매한 프롬프트로 인해 모델이 형식 준수에 어려움을 겪고 있다. 

아래는 이제 위 COT를 쓸 경우 생성되는 System 프롬프트와 User 프롬프트이다.
- System : Your input fields are:
request (str): Your output fields are:
reasoning (str):
answer (str): All interactions will be structured in the following way, with the appropriate values filled in.
[[ ## request ## ]] {request}
[[ ## reasoning ## ]] {reasoning}
[[ ## answer ## ]] {answer}
[[ ## completed ## ]] In adhering to this structure, your objective is: Given the fields request, produce the fields answer.
- User : [[ ## request ## ]] CoT 방법에 대해서 단계적으로 자세히 설명해 주세요.
Respond with the corresponding output fields, starting with the field [[ ## reasoning ## ]], then [[ ## answer ## ]], and then ending with the marker for [[ ## completed ## ]].

### 문제 2 -> 모듈 / 시그니처와 겹치는 EXAONE의 chat template.
- 원인 : Exaone의 입력 인자를 조절함으로써 항상 `think`를 활성화 하도록 구성하였는데, DSPy의 CoT 모듈과, 시그니처를 응용해서 사용할 경우 모델이 제대로 형식을 준수하지 못하고, 의도한 대로 reasoning도 하지 못해 여러 방면으로 문제가 발생한다.
- 해결 방안 탐구 
    - 먼저, 현상 탐구부터 해 보자면, Exaone 모델은 '거의 모든' 경우에 대해서 `</think>` 전까지 제대로 reasoning을 하도록 설계되어 있다.
    - 즉, 해당 토큰 전까지를 이제 reasoning 단계로 설정할 수 있을 것이다. 
    - 또한, 해당 토큰 이후를 이제 answer 단계로 설정할 수 있을 것이다.
    - 그런데 의문점이, 왜 `[[ ## completed ## ]]` 마커가 존재하는 것이지? 이는 endoftoken으로 구성하는 것일까? 직관적으로 이해가 잘 안 된다.
    - 최대한 Exaone의 기존 형식에 벗어나지 않게 구성하여 형식 준수 강제로 인한 성능 저하 현상을 없애보도록 하자.
    - 정리하자면 아래와 같이 해결 방안을 구성할 수 있겠다. 
    1. Exaone 모델에 맞는 시그니처 구성. 이때, LeeT 문제 풀이가라는 사실을 명시하도록 하자.
    2. 이 시그니처를 바탕으로, Exaone 모델에 맞는 모듈을 구성하자. 
    3. 추가적으로, 굳이 `think` 안에서는 따로 지켜야 할 형식은 없도록 하고, answer 단계 부분에서만 이제 최종 정답이 정수로 잘 파싱되도록 구성하는 게 좋을 것이다.
    
    번외) Predict를 사용하면 어떨까?

In [24]:
PREDICT = dspy.Predict("request -> answer")

In [ ]:
with mlflow.start_run(run_name="predict_single_test"):
    test_result = PREDICT(**{"request" : "CoT 방법에 대해서 단계적으로 자세히 설명해 주세요."})
    # 이렇게 **를 사용해야 한다. 그래야 이러한 request 라고 시그니처에 지정된 입력 인자와 대응이 되는 것을 알 수 있다.

🏃 View run predict_single_test at: http://127.0.0.1:5001/#/experiments/366534923737239846/runs/4114966b416b4b07892e836d7bc12681
🧪 View experiment at: http://127.0.0.1:5001/#/experiments/366534923737239846


Trace(trace_id=tr-0ca2196be3534a12c39b52ed007e77fd)

In [47]:
test_result

Prediction(
    answer='Chain-of-Thought (CoT) 방법은 복잡한 문제를 해결할 때 단계별로 사고를 전개하는 방법입니다. 이 방법은 모델이 문제를 작은 단계로 나누고 각 단계에서 추론을 수행함으로써 최종 답에 도달하는 과정을 말합니다.\n\n단계별 설명:\n1. 문제 이해: 먼저 주어진 문제를 정확히 이해합니다.\n2. 단계 분할: 문제를 해결하기 위해 필요한 하위 단계들로 나눕니다.\n3. 단계별 추론: 각 하위 단계에 대해 차례로 추론을 수행합니다. 이때 중간 결과를 기록합니다.\n4. 통합: 모든 하위 단계의 결과를 종합하여 최종 답을 도출합니다.\n\n예시: "사과 3개와 배 2개가 있을 때 과일은 총 몇 개인가?"라는 질문에 대해,\n- 단계 1: 사과의 개수는 3개.\n- 단계 2: 배의 개수는 2개.\n- 단계 3: 총 과일 수 = 사과 개수 + 배 개수 = 3 + 2 = 5.\n\nCoT의 장점:\n- 복잡한 문제를 체계적으로 해결할 수 있습니다.\n- 추론 과정이 투명해져 오류를 찾기 쉽습니다.\n- 모델의 성능을 향상시킬 수 있습니다.'
)

In [49]:
lm.history[-1]['messages']

[{'role': 'system',
  'content': 'Your input fields are:\n1. `request` (str):\nYour output fields are:\n1. `answer` (str):\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## request ## ]]\n{request}\n\n[[ ## answer ## ]]\n{answer}\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        Given the fields `request`, produce the fields `answer`.'},
 {'role': 'user',
  'content': '[[ ## request ## ]]\nCoT 방법에 대해서 단계적으로 자세히 설명해 주세요.\n\nRespond with the corresponding output fields, starting with the field `[[ ## answer ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.'}]

In [ ]:
lm.history[-1]['response'].choices[0].message.content
# lm.history를 응용해서 결과를 확인해 볼 수 있다. 

'We are given a request: "CoT 방법에 대해서 단계적으로 자세히 설명해 주세요."\n We need to produce an answer that explains the Chain-of-Thought (CoT) method step by step in detail.\n Then we must output the answer in the specified format.\n\n Steps for explaining CoT:\n 1. Introduction: Define what CoT is.\n 2. Why it\'s used: Explain the motivation (e.g., breaking down complex problems).\n 3. How it works: Describe the step-by-step reasoning process.\n 4. Example: Provide a simple example to illustrate.\n 5. Benefits: List the advantages of using CoT.\n\n However, note that the request is in Korean, so we should respond in Korean as well.\n\n Let\'s structure the answer:\n\n[[ ## answer ## ]]\nChain-of-Thought (CoT) 방법은 복잡한 문제를 해결할 때 단계별로 사고를 전개하는 방법입니다. 이 방법은 모델이 문제를 작은 단계로 나누고 각 단계에서 추론을 수행함으로써 최종 답에 도달하는 과정을 말합니다.\n\n단계별 설명:\n1. 문제 이해: 먼저 주어진 문제를 정확히 이해합니다.\n2. 단계 분할: 문제를 해결하기 위해 필요한 하위 단계들로 나눕니다.\n3. 단계별 추론: 각 하위 단계에 대해 차례로 추론을 수행합니다. 이때 중간 결과를 기록합니다.\n4. 통합: 모든 하위 단계의 결과를 종합하여 최종 답을 도출합니다.\n\n예시: "사

```
'We are given a request: "CoT 방법에 대해서 단계적으로 자세히 설명해 주세요."\n We need to produce an answer that explains the Chain-of-Thought (CoT) method step by step in detail.\n Then we must output the answer in the specified format.\n\n Steps for explaining CoT:\n 1. Introduction: Define what CoT is.\n 2. Why it\'s used: Explain the motivation (e.g., breaking down complex problems).\n 3. How it works: Describe the step-by-step reasoning process.\n 4. Example: Provide a simple example to illustrate.\n 5. Benefits: List the advantages of using CoT.\n\n However, note that the request is in Korean, so we should respond in Korean as well.\n\n Let\'s structure the answer:\n\n[[ ## answer ## ]]\nChain-of-Thought (CoT) 방법은 복잡한 문제를 해결할 때 단계별로 사고를 전개하는 방법입니다. 이 방법은 모델이 문제를 작은 단계로 나누고 각 단계에서 추론을 수행함으로써 최종 답에 도달하는 과정을 말합니다.\n\n단계별 설명:\n1. 문제 이해: 먼저 주어진 문제를 정확히 이해합니다.\n2. 단계 분할: 문제를 해결하기 위해 필요한 하위 단계들로 나눕니다.\n3. 단계별 추론: 각 하위 단계에 대해 차례로 추론을 수행합니다. 이때 중간 결과를 기록합니다.\n4. 통합: 모든 하위 단계의 결과를 종합하여 최종 답을 도출합니다.\n\n예시: "사과 3개와 배 2개가 있을 때 과일은 총 몇 개인가?"라는 질문에 대해,\n- 단계 1: 사과의 개수는 3개.\n- 단계 2: 배의 개수는 2개.\n- 단계 3: 총 과일 수 = 사과 개수 + 배 개수 = 3 + 2 = 5.\n\nCoT의 장점:\n- 복잡한 문제를 체계적으로 해결할 수 있습니다.\n- 추론 과정이 투명해져 오류를 찾기 쉽습니다.\n- 모델의 성능을 향상시킬 수 있습니다.\n\n[[ ## completed ## ]]\n</think>\n\n[[ ## answer ## ]]\nChain-of-Thought (CoT) 방법은 복잡한 문제를 해결할 때 단계별로 사고를 전개하는 기법입니다. 다음은 세부 단계별 설명입니다:\n\n1. **문제 분해**:  \n   주어진 문제를 작은 하위 작업들로 분할합니다.  \n   *예시: "두 수의 합을 구하라" → "첫 번째 수 확인", "두 번째 수 확인", "덧셈 수행"*\n\n2. **중간 추론 생성**:  \n   각 하위 작업마다 논리적 추론을 단계별로 기록합니다.  \n   *예시: "5 + 7" → "5는 5", "7은 7", "5+7=12"*\n\n3. **연결성 강화**:  \n   이전 단계의 결과를 다음 단계의 입력으로 활용하며 사고 흐름을 자연스럽게 연결합니다.  \n   *예시: "5를 확인 → 7을 확인 → 5+7 계산"*\n\n4. **검증 단계**:  \n   각 추론 결과를 검토하며 오류 가능성을 줄입니다.  \n   *예시: "5+7이 13이 아닐까? → 5+7=12로 재확인"*\n\n5. **최종 통합**:  \n   모든 중간 결과를 종합해 최종 답변을 도출합니다.  \n   *예시: "중간 결과 12를 종합 → 답: 12"*\n\n**CoT 핵심 목적**:  \n- 복잡한 문제를 체계적으로 해결  \n- 추론 과정의 투명성 확보  \n- 모델의 추론 능력 향상  \n\n[[ ## completed ## ]]'
```
마찬가지로, 뒤죽박죽 섞였으며, 중복되어 제시되었다는 사실을 확인할 수 있다. 중복되면 항상 앞 부분을 가져다 쓰는 것 같다.

### 문제 2 해결 시작 - 먼저 시그니처부터 만들어서 적용해보기.

In [121]:
class ExaoneLegal(dspy.Signature):
    """Leet 문제를 정확히 맞춰야 한다."""
    passage = dspy.InputField(desc="법률 관련 지문")
    question = dspy.InputField(desc="분석할 질문") 
    options = dspy.InputField(desc="선택지")
    statements = dspy.InputField(desc="관련 보기")
    
    # 단순화된 출력 - thinking은 모델에게 맡기고 최종 답만 구조화
    final_answer : int = dspy.OutputField(
        desc="1~5 사이의 정수로 구성된 최종 답안 번호. 맨 마지막에 한 번만 제시하면 된다.",
        format=int
    )

이후, 이제 시그니처를 Leet 문제 형식에 맞게 구성했으니 파싱하고 입력을 통해 관찰해 보자.

In [122]:
PREDICT_EXAONE_LEGAL = dspy.Predict(ExaoneLegal)

In [123]:
PREDICT_EXAONE_LEGAL

Predict(ExaoneLegal(passage, question, options, statements -> final_answer
    instructions='Leet 문제를 정확히 맞춰야 한다.'
    passage = Field(annotation=str required=True json_schema_extra={'desc': '법률 관련 지문', '__dspy_field_type': 'input', 'prefix': 'Passage:'})
    question = Field(annotation=str required=True json_schema_extra={'desc': '분석할 질문', '__dspy_field_type': 'input', 'prefix': 'Question:'})
    options = Field(annotation=str required=True json_schema_extra={'desc': '선택지', '__dspy_field_type': 'input', 'prefix': 'Options:'})
    statements = Field(annotation=str required=True json_schema_extra={'desc': '관련 보기', '__dspy_field_type': 'input', 'prefix': 'Statements:'})
    final_answer = Field(annotation=int required=True json_schema_extra={'desc': '1~5 사이의 정수로 구성된 최종 답안 번호. 맨 마지막에 한 번만 제시하면 된다.', 'format': <class 'int'>, '__dspy_field_type': 'output', 'prefix': 'Final Answer:'})
))

In [108]:
def load_prompts_with_indices(file_path: str) -> list:
    """단일 CSV 파일에서 프롬프트를 로드하고, 추적을 위해 행 인덱스를 함께 반환합니다."""
    prompts_with_indices = []
    if os.path.exists(file_path) and file_path.endswith(".csv"):
        df = pd.read_csv(file_path)
        for idx, row in df.iterrows():
            prompt_parts = []
            if pd.notna(row.get("지문")):
                prompt_parts.append(f"[passage]\n{row['지문']}")
            if pd.notna(row.get("질문")):
                prompt_parts.append(f"[question]\n{row['질문']}")
            if pd.notna(row.get("보기")):
                prompt_parts.append(f"[list of statements]\n{row['보기']}")
            if pd.notna(row.get("선택지")):
                prompt_parts.append(f"[option]\n{row['선택지']}")
            if pd.notna(row.get("label")):
                prompt_parts.append(f"[label]\n{row['label']}")

            prompt = "\n\n".join(prompt_parts)
            if prompt:
                prompts_with_indices.append((idx, prompt))
    return prompts_with_indices

In [109]:
csv_files = glob.glob(os.path.join(DATA_DIRECTORY, "*.csv"))
list_prompts_with_indices = []
for csv_file in csv_files :
    prompts_with_indices = load_prompts_with_indices(csv_file)
    list_prompts_with_indices.append(prompts_with_indices)

In [110]:
import re

def extract_year_from_path(file_path):
    """파일 경로에서 연도를 추출합니다."""
    year_pattern = r'(20[0-9]{2})'
    match = re.search(year_pattern, file_path)
    return int(match.group(1)) if match else None

def create_year_based_dataset(csv_files, list_prompts_with_indices):
    """
    CSV 파일들을 연도별로 분리하여 데이터셋을 생성합니다.
    2024년 -> 훈련용, 2025년 -> 테스트용
    """
    train_data = []  # 2024년 데이터
    test_data = []   # 2025년 데이터
    
    for file_idx, (csv_file, prompts_with_indices) in enumerate(zip(csv_files, list_prompts_with_indices)):
        year = extract_year_from_path(csv_file)
        print(f"파일: {csv_file} -> 연도: {year}")
        
        if year == 2024:
            train_data.extend(prompts_with_indices)
        elif year == 2025:
            test_data.extend(prompts_with_indices)
        else:
            print(f"경고: 알 수 없는 연도 {year} (파일: {csv_file})")
    
    print(f"\n2024년 데이터 (훈련용): {len(train_data)}개")
    print(f"2025년 데이터 (테스트용): {len(test_data)}개")
    
    return train_data, test_data

# 연도별 데이터 분리
train_prompts, test_prompts = create_year_based_dataset(csv_files, list_prompts_with_indices)

파일: /home/eoeldroal/WorkPlace/Legal/Legal-DSPy/Data/LU/언어이해_2024.csv -> 연도: 2024
파일: /home/eoeldroal/WorkPlace/Legal/Legal-DSPy/Data/LU/언어이해_2025.csv -> 연도: 2025

2024년 데이터 (훈련용): 30개
2025년 데이터 (테스트용): 30개


In [111]:
def create_dspy_examples_from_prompts(prompts_with_indices):
    """
    프롬프트 데이터를 DSPy Example 객체로 변환합니다.
    
    Args:
        prompts_with_indices: (인덱스, 프롬프트) 튜플들의 리스트
    
    Returns:
        list: DSPy Example 객체들의 리스트
    """
    examples = []
    
    for idx, prompt in prompts_with_indices:
        # 프롬프트를 섹션별로 파싱
        sections = {}
        current_section = None
        current_content = []
        
        for line in prompt.split('\n'):
            if line.startswith('[') and line.endswith(']'):
                # 이전 섹션 저장
                if current_section:
                    sections[current_section] = '\n'.join(current_content).strip()
                # 새 섹션 시작
                current_section = line[1:-1]  # [] 제거
                current_content = []
            elif current_section:
                current_content.append(line)
        
        # 마지막 섹션 저장
        if current_section:
            sections[current_section] = '\n'.join(current_content).strip()
        
        # DSPy Example 생성
        example_data = {
            'passage': sections.get('passage', ''),
            'question': sections.get('question', ''),
            'options': sections.get('option', ''),
            'statements': sections.get('list of statements', ''),
            'answer': sections.get('label', ''),
            'original_index': idx
        }
        
        # DSPy Example 생성 - passage, question, options, statements를 입력으로 설정
        input_fields = ['passage', 'question']
        if 'options' in example_data:
            input_fields.append('options')
        if 'statements' in example_data:
            input_fields.append('statements')
            
        example = dspy.Example(**example_data).with_inputs(*input_fields)
        examples.append(example)
    
    return examples

# 연도별 DSPy 데이터셋 생성
train_set = create_dspy_examples_from_prompts(train_prompts)
test_set = create_dspy_examples_from_prompts(test_prompts)

print(f"훈련 데이터셋 (2024년): {len(train_set)}개")
print(f"테스트 데이터셋 (2025년): {len(test_set)}개")

print("\n훈련 데이터 첫 번째 예시:")
if train_set:
    example = train_set[0]
    print(f"입력 필드: {example.inputs()}")
    print(f"정답: {example.answer if 'answer' in example else 'N/A'}")

print("\n테스트 데이터 첫 번째 예시:")
if test_set:
    example = test_set[0]
    print(f"입력 필드: {example.inputs()}")
    print(f"정답: {example.answer if 'answer' in example else 'N/A'}")

훈련 데이터셋 (2024년): 30개
테스트 데이터셋 (2025년): 30개

훈련 데이터 첫 번째 예시:
입력 필드: Example({'passage': "규범교의적 학문을 자처하는 법학은 학문성에 관한 논쟁에 시달려 왔다. 입법자의 권력 행사로 법전의 한마디가 바뀌면, 오랫동안 가꾼 해석의 축적이 순식간에 무용지물이 되기 때문이다. 이에 대한 도전으로서 알베르트는 경험적 반증가능성을 강조하는 비판적 합리주의에 입각하여 법학의 학문성을 새롭게 이해하고자 한다. \n알베르트는 우선 법학의 은폐된 특징을 신학과의 비교를 통해 문제 삼는다. 법학은 당국의 고시에서 진리를 얻어내는 점에서 신학과 구조적 유사성을 가지기 때문이다. 신학이 경전의 해석을 통해 권위를 확보하듯, 법학은 법전을 확인하고 문제 해결과 관련하여 이를 해석한다. 이때 경전이나 법전은 학문적 비판이나 성찰의 대상이 아니라 해석적 권위의 원천이자 근거가 될 따름이다. 그가 보기에 법학이 신학과의 구조적 유사성을 탈피하려면, 해석에서 자연법이냐 사회학이나의 양자택일을 감수해야 한다. 선택의 결과는 자명하다. 절대성을 가진 규범적 현실에 의해 실정법이 구성되고 또 구속된다고 보는 견해는 신적인 힘으로 설립된 세계를 믿는 관점에 의해서만 유지될 수 있기 때문이다. 알베르트는 법을 인간의 문화적 성취로 간주하고, 사회적 삶의 사실 중 사회 구성원의 상호 행위 조종의 영역에 속하는 것으로 본다. \n물론 이 경우에도 법을 현실주의적으로 보느냐, 규범주의적으로 보느냐의 문제는 남는다. 알베르트는 법을 사회적 사실로, 법학을 경험과학으로 볼 것을 주장한다. 그에 따르면 규범에 관한 법학적 언명은 규범 자체와 다르게 규범성이 없으며, 이 구별을 무시한다면 규범의 인식적 파악이라는 이념은 사라지게 된다. 그는 법률 문언의 규범성은 인정하지만, 그 문언에 관하여 의미를 밝히는 법학은 다르다고 말한다. \n법학에 대한 알베르트의 현실주의적 파악에는 곤란해 보이는 점도 있다. 예컨대, 법률 문언에 흠결이 존재하여 적극적

In [112]:
sample_example = test_set[0]  # 첫 번째 테스트 데이터 사용

In [113]:
if test_set:
    print("=== Custom Legal 테스트 실행 ===")
    
    # 입력 준비
    inputs = {
        'passage': sample_example.passage,
        'question': sample_example.question,
        'options': sample_example.options,
        'statements': sample_example.statements
    }
    
    print(f"입력 준비 완료")
    
    # MLflow 실행과 함께 추론 수행
    with mlflow.start_run(run_name="cot_single_test"):
        try:
            # Chain of Thought 실행
            result = PREDICT_EXAONE_LEGAL(**inputs)
            
            print("\\n=== 결과 ===")
            print(f"\\n모델 답: {result.final_answer}")
            print(f"정답: {sample_example.answer}")
            print(f"정답 일치: {str(result.final_answer).strip() == str(sample_example.answer).strip()}")
            
        except Exception as e:
            print(f"오류 발생: {e}")
            import traceback
            traceback.print_exc()
else:
    print("테스트할 데이터가 없습니다.")

=== Custom Legal 테스트 실행 ===
입력 준비 완료
\n=== 결과 ===
\n모델 답: 2
정답: 2
정답 일치: True
🏃 View run cot_single_test at: http://127.0.0.1:5001/#/experiments/366534923737239846/runs/072fe3032bfe42f98b9dd50ae6a4b35e
🧪 View experiment at: http://127.0.0.1:5001/#/experiments/366534923737239846


Trace(trace_id=tr-a2efa7b4d92ff00fd78a934c290f547f)

In [114]:
lm.history[-1]['response'].choices[0].message.content

'First, I need to answer the question: "윗글의 내용과 일치하지 않는 것은?" which means "Which of the following does not match the content of the passage?" I have to choose the incorrect option from ① to ⑤.\n\nThe passage is about how literature reflects society, especially in the context of law and crime novels. It discusses the Newgate novels in 1830s England, their role in advocating for penal reform, and how they portrayed criminals and justice.\n\nNow, let\'s evaluate each option one by one based on the passage.\n\nOption ①: "형법 개혁 운동은 범죄소설 열풍의 계기이자 성과였다." Translation: "The penal reform movement was both the cause and the achievement of the crime novel craze."\n\nFrom the passage: "1830년대 영국에서 유행한 범죄소설은 이러한 법과 문학의 상호작용을 잘 보여 준다." (The crime novels popular in 1830s England well illustrate the interaction between law and literature.) Also, "그것은 동시에 당대의 지배적 범죄 담론에 대한 대항 담론을 선전․유포하여 형법 개혁의 원동력이 되기도 했다." (It simultaneously spread a counter-discourse against the dominant crime discourse of the time, b

출력 결과는 아래와 같다.
```
'First, I need to answer the question: "윗글의 내용과 일치하지 않는 것은?" which means "Which of the following does not ....
and willpower – supported.\n\nSo, option ② is the one that doesn\'t match.\n\nTherefore, the answer should be ②.\n\nNow, for the output, I need to provide `[[ ## final_answer ## ]]` with the integer, and end with `[[ ## completed ## ]]`.\n\nSo, final_answer is 2.\n</think>\n\n[[ ## final_answer ## ]]\n2'
```
아주 이상적이게 잘 나왔다.
- `think`안에서는 이제 형식을 지켜라고 노력하지 않았다.
- 하지만 마지막 부분에서는 형식을 '최종 응답' 때 지켜야 한다는 사실을 상기했다.
- 또한, `think`가 끝나고 최종 응답 때는 이제 정답에 해당하는 숫자 하나만 제시했다.
- 이때, 나중에 혹시나 어느 정도 설명 가능성이나 사용자의 요구에 맞추기 위해 이러한 최종 응답 부분에 간단히 설명은 해야 한다고 제시해야 하나?
    - 나중에 생각하자.

### 문제 2 해결 완료

In [117]:
def validate_answer(example, pred, trace=None):
    return int(example.answer) == int(pred.final_answer)

In [124]:
# 코드에서 재사용할 수 있는 평가기 설정
evaluator = Evaluate(devset=test_set, num_threads=144, display_progress=True, display_table=30)
with mlflow.start_run(run_name="Cot_before_optimization"):
    # 평가 실행
    evaluator(PREDICT_EXAONE_LEGAL, validate_answer)

Average Metric: 14.00 / 14 (100.0%):  47%|████▋     | 14/30 [02:02<00:57,  3.61s/it]

2025/08/09 02:12:35 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Average Metric: 23.00 / 25 (92.0%):  83%|████████▎ | 25/30 [03:26<00:48,  9.72s/it] 

2025/08/09 02:13:57 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Average Metric: 25.00 / 29 (86.2%): 100%|██████████| 30/30 [05:58<00:00, 37.65s/it]

...

Average Metric: 25.00 / 30 (83.3%): : 36it [11:34, 19.29s/it]

2025/08/09 02:22:03 INFO dspy.evaluate.evaluate: Average Metric: 25 / 30 (83.3%)


,passage,question,options,statements,answer,original_index,final_answer,validate_answer
0,문학이 사회와 그 구성원의 삶을 반영한다는 명제는 법의 영역에도 적용된다. 문학적 서사는 한 시대의 법인식과 정의관을 ...,윗글의 내용과 일치하지 않는 것은?,"①형법 개혁 운동은 범죄소설 열풍의 계기이자 성과였다. ②뉴게이트 소설은 범죄를 질병으로, 형벌을 치료로 이해한 당대 ...",,2,0,2,✔️ [True]
1,문학이 사회와 그 구성원의 삶을 반영한다는 명제는 법의 영역에도 적용된다. 문학적 서사는 한 시대의 법인식과 정의관을 ...,㉠∼㉤에 대한 이해로 적절하지 않은 것은?,①㉠은 죄에 비해 과한 형을 구형하거나 사형 선고를 남발하는 현상을 가리킨다. ②㉡은 살아남기 위해 주어진 계급적 위치...,,2,1,2,✔️ [True]
2,문학이 사회와 그 구성원의 삶을 반영한다는 명제는 법의 영역에도 적용된다. 문학적 서사는 한 시대의 법인식과 정의관을 ...,윗글에서 추론한 것으로 가장 적절한 것은?,①디킨스는 법의 부조리에 대한 비판과 범죄의 해악에 대한 훈계를 한 작품에서 동시에 수행할 수 없다고 보았을 것이다. ...,,5,2,5,✔️ [True]
3,동서양의 전설에 나오는 귀신 중 흡혈귀는 문학의 소재로 오래 활용되었다. 특히 흡혈귀는 슬라브 또는 헝가리의 전설에 자...,윗글의 내용과 일치하지 않는 것은?,①코프로포르피리노젠I은 포르피린의 한 종류이다. ②미오글로빈과 시토크롬은 헴을 보철그룹으로 가지고 있는 단백질이다. ③...,,4,3,4,✔️ [True]
4,동서양의 전설에 나오는 귀신 중 흡혈귀는 문학의 소재로 오래 활용되었다. 특히 흡혈귀는 슬라브 또는 헝가리의 전설에 자...,윗글에서 추론한 내용으로 가장 적절한 것은?,①콕스는 조지 3세의 모발에서 비소 대사와 관련된 효소 유전자의 결함을 찾고자 하였을 것이다. ②조지 3세는 불면증과 ...,,2,4,2,✔️ [True]
5,동서양의 전설에 나오는 귀신 중 흡혈귀는 문학의 소재로 오래 활용되었다. 특히 흡혈귀는 슬라브 또는 헝가리의 전설에 자...,"<보기>를 바탕으로 본문을 이해할 때, 가장 적절한 것은?","①효소 ㉠, ㉡의 산물은 도스포르피린증 환자의 체내에 축적될 것이다. ②효소 ㉢의 산물이 코프로포르피리노젠I로 전환되는...",<보 기> 헴 합성은 (가)와 같은 다단계 효소 촉매 과정에 의하여 일어난다. 효소는 ‘기질’의 화학적 구조를 변화시키...,3,5,3,✔️ [True]
6,"기존의 역사가들이 민주주의, 노예제와 같은 정치․사회제도의 모델을 찾기 위해 고대 그리스와 로마에 주목했다면, 최근에는...",윗글에 대한 이해로 적절하지 않은 것은?,①플라톤은 파이데라스티아의 대상을 일정한 지적 성장 단계의 소년으로 한정했다. ②크세노폰은 에라스테스를 소년의 육체를 ...,,2,6,2,✔️ [True]
7,"기존의 역사가들이 민주주의, 노예제와 같은 정치․사회제도의 모델을 찾기 위해 고대 그리스와 로마에 주목했다면, 최근에는...",윗글로 보아 다음 설명 중 가장 적절한 것은?,①아테네와 스파르타에서는 모두 이십 대 청년이 에로메노스에서 배제되었다. ②아테네에서와 달리 스파르타에서의 에라스테스는...,,3,7,3,✔️ [True]
8,"기존의 역사가들이 민주주의, 노예제와 같은 정치․사회제도의 모델을 찾기 위해 고대 그리스와 로마에 주목했다면, 최근에는...","윗글과 <보기>를 연결하여 평가할 때, 가장 적절한 것은?",①도자기에 그려진 장면은 에로메노스와 에라스테스 관계에 대한 골든의 해석과 상충하는군. ②그리스 도자기의 소년애 장면은...,<보 기> 고대 그리스 도자기에 묘사된 소년애 장면은 이성애 장면에 비해 훨씬 덜 노골적이다. 이는 자유민 소년이 성적...,2,8,2,✔️ [True]
9,사법심사는 다수주의의 예외로 간주되기도 한다. 민주적 절차로 선출된 의회나 행정부의 결정이 합헌 여부를 기준으로 무효화...,네 가지 모델에 대한 설명으로 가장 적절한 것은?,①긍정적 반응 모델은 연방대법원의 전문성과 공정성에 대한 대중의 불신을 반영한다. ②연방대법원의 결정을 여론이 즉각 수...,,3,9,3,✔️ [True]


🏃 View run Cot_before_optimization at: http://127.0.0.1:5001/#/experiments/366534923737239846/runs/359c46db64554095854229d493d49821
🧪 View experiment at: http://127.0.0.1:5001/#/experiments/366534923737239846


[Trace(trace_id=tr-724fee5b401e2d68660bf1a440da6cf5), Trace(trace_id=tr-ffb39e67f4e2a5e1aa323118b68ec50a), Trace(trace_id=tr-12edab40c8ae20f5983226ba070f416d), Trace(trace_id=tr-4ed3ad7ce597caad5460a84613b14aba), Trace(trace_id=tr-b13adfbfdc6c36196e62283df13e3faa), Trace(trace_id=tr-31e518cb72a535b83411bf5185dac37f), Trace(trace_id=tr-8f9a852c2b3c7d60b1576ab648d48f30), Trace(trace_id=tr-c5b82b19cecdf13b8cb814eba0b5492b), Trace(trace_id=tr-b448a50de0f4b50eebfa65fa42f1859a), Trace(trace_id=tr-0abac5f1648880a18dcc979b748009b2)]

```
2025/08/09 02:06:11 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=4000. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.6)  if the reason for truncation is repetition.
```
- Truncated 문제가 발생한다. 아무래도 think까지 모두 하다 보니 사용하는 토큰이 기본적으로 많은 것 같다. 따라서 max_tokens 수를 늘려 보자.
- 16384로 늘리고 다시 돌렸다. 
- 결과: 기존 초기 정답률 50%보다 약간 나은 수준에서, 86%까지 끌어올렸다. think 활성화 + DSPy 양식과의 융합 덕택이다.

In [ ]:
THREADS = 144

kwargs = dict(num_threads=THREADS, teacher_settings=dict(lm=o4_mini), prompt_model=lm)
optimizer = dspy.MIPROv2(metric=validate_answer, auto="medium", **kwargs)

kwargs = dict(requires_permission_to_run=False, max_bootstrapped_demos=4, max_labeled_demos=4)
optimized_module = optimizer.compile(PREDICT_EXAONE_LEGAL, trainset=train_set, **kwargs)

2025/08/08 19:25:02 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 18
minibatch: False
num_fewshot_candidates: 12
num_instruct_candidates: 6
valset size: 24

2025/08/08 19:25:02 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/08/08 19:25:02 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/08/08 19:25:02 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=12 sets of demonstrations...


Bootstrapping set 1/12
Bootstrapping set 2/12
Bootstrapping set 3/12


 17%|█▋        | 1/6 [00:30<02:30, 30.08s/it]/home/eoeldroal/miniconda3/envs/DSPy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 6/6 [02:08<00:00, 21.36s/it]


Bootstrapped 3 full traces after 5 examples for up to 1 rounds, amounting to 6 attempts.
Bootstrapping set 4/12


 67%|██████▋   | 4/6 [01:07<00:36, 18.42s/it]2025/08/08 19:29:12 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/08/08 19:29:57 ERROR dspy.teleprompt.bootstrap: Failed to run or to evaluate example Example({'passage': '금융, 마케팅, 의료 등 다양한 분야에서 생성되는 빅데이터는 많은 경우 개인정보를 포함하고 있어 데이터를 활용하는 과정에서 민감한 개인정보가 유출될 가능성이 있다. 따라서 빅데이터 구축 과정에서 개인정보의 전부 또는 일부를 삭제하거나 대체함으로써 개인의 신원이 드러나지 않도록 하면서도 해당 데이터의 활용성을 최대한 유지할 수 있도록 하는 개인정보 비식별화 기술을 사용한다. \n데이터 집합에서 정보를 표현하는 최소 단위를 속성이라고 하고 다양한 속성들의 조합으로 표현된 하나의 정보를 레코드라고 한다. 데이터 집합은 이 레코드들의 집합이다. 비식별화 기술은 속성을 식별자, 준식별자, 일반속성, 민감속성으로 구분한다. 주민번호와 같이 그 자체만으로도 누구인지 식별 가능한 속성이 식별자이다. 반면에 성별, 연령, 주소와 같이 개인에 대한 직접적인 식별은 불가능하지만 이들 속성이 결합하면 개인에 대한 식별이 가능해지는 속성을 준식별자라고 한다. 성별, 이름, 연령으로 구성되어 있는 원본 데이터 집합이 있을 때, 이름에서 성씨만을 남겨 비식별 데이터 집합을 만들었다고 하자. 비록 이름은 성만 남기고 가려져 있지만 \'남성\'이 유일하거나, 성이 \'이씨\'이면서 \'35세인 사람이 유일하다면, 원본에 이 두 사람이 포함된 사실을 알면서 이들 각자의 유일한 속성값 조합을 미리 알고 있는 사람은 특정 개인을 재식별할 수 있다. 일반적으로 개인정보는 개인의 여러 속성과 결합하여 사용된다

...

2025/08/08 19:52:29 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Average Metric: 10.00 / 19 (52.6%):  83%|████████▎ | 20/24 [00:37<00:04,  1.10s/it]

2025/08/08 19:52:42 ERROR dspy.utils.parallelizer: Error for Example({'passage': '투표 참여에 대한 설명은 유권자가 투표에 참여하기 위해 치르는 비용에 주목한다. 예를 들어 투표소가 거주지와 가깝거나 이동하기 쉬운 곳에 있을수록 유권자들이 더 쉽게 투표할 수 있다. 또한 투표 참여 비용의 큰 부분을 차지하는 것이 <가> 선거와 후보에 대한 정보를 획득하고 처리하는 비용 </가>이다. 일반적으로 사회경제적 지위가 높은 유권자들이 그렇지 않은 유권자들에 비해 더 열심히 투표에 참여하는 이유는 전자가 이러한 비용을 더 낮게 체감하기 때문이다. \n선거일 날씨도 투표 참여를 결정하는 데 있어 비용의 구성 요소가 될 수 있다. 비가 오는 날에는 투표소에 가거나 줄을 서서 차례를 기다리는 것이 불편한 일이기 때문이다. 따라서 기존 연구들은 궂은 날씨가 유권자가 투표하러 가는 것을 망설이게 한다는 데 동의한다. 다만 지금까지 학문적 관심의 초점은 궂은 날씨로 인 한 비용 증가가 실제로 투표율을 낮출 만큼 큰 문제인가에 맞춰져 왔다. 어떤 학자들은 날씨가 유발하는 비용 증가는 미미하다고 주장하지만, 다른 학자들은 작은 불편으로 인한 추가 비용도 상당수 유권자 사이에서 투표와 기권의 선택을 뒤바꿀 수 있다고 본다. \n미국 대통령선거를 대상으로 한 최근 연구에 따르면 주 단위에서 강수량과 투표율을 비교했을 때, 강수량이 평년보다 1인치 증가할 때 투표율은 약 2.4% 포인트 감소했다. 다만 이 연구는 <나> 주별 강수량을 측정하기 위해 그 주에서 가장 큰 도시의 선거 당일 강수량을 대리지표(proxy)로 활용 </나> 했다는 점에서 비판의 대상이 되었다. 그러나 이러한 문제를 교정한 다른 연구에서도 강수량의 증가가 투표율 감소를 가져온다는 증거가 제시되었다. \n그런데 투표와 관련된 비용에는 투표에 참여하는 데 필요한 직접비용뿐 아니라, <다> 투표에 참여하느라 다른 선택을 포기하는 데서 오는 기회비용 </다>도

Average Metric: 12.00 / 22 (54.5%): 100%|██████████| 24/24 [00:43<00:00,  1.83s/it]

2025/08/08 19:52:45 INFO dspy.evaluate.evaluate: Average Metric: 12.0 / 24 (50.0%)
2025/08/08 19:52:45 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 50.0 with parameters ['Predictor 0: Instruction 3', 'Predictor 0: Few-Shot Set 11'].
2025/08/08 19:52:45 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [37.5, 45.83, 41.67, 50.0, 45.83, 58.33, 41.67, 50.0, 50.0, 58.33, 58.33, 58.33, 50.0, 58.33, 50.0, 54.17, 50.0, 50.0, 50.0]
2025/08/08 19:52:45 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 58.33
2025/08/08 19:52:45 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/08/08 19:52:45 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 58.33!


In [80]:
optimized_module

predict = Predict(StringSignature(passage, question, options, statements -> reasoning, answer
    instructions='주어진 법률 문서를 분석하고, 질문에 대한 정답을 선택하기 위해 단계별 추론을 수행하세요. 특히, 알베르트와 사비니의 주장을 비교하면서 각 옵션의 적절성을 평가해야 합니다. 정답을 선택할 때는 문서의 내용과 정확히 일치하는지 확인하고, 모순이 있는 부분을 주의 깊게 검토하세요.'
    passage = Field(annotation=str required=True json_schema_extra={'desc': '법률 관련 지문', '__dspy_field_type': 'input', 'prefix': 'Passage:'})
    question = Field(annotation=str required=True json_schema_extra={'desc': '지문에 대한 질문', '__dspy_field_type': 'input', 'prefix': 'Question:'})
    options = Field(annotation=str required=True json_schema_extra={'desc': '선택지', '__dspy_field_type': 'input', 'prefix': 'Options:'})
    statements = Field(annotation=str required=True json_schema_extra={'desc': '보기', '__dspy_field_type': 'input', 'prefix': 'Statements:'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_

In [82]:
with mlflow.start_run(run_name="Cot_after_optimization"):
    evaluator(optimized_module, validate_answer)

Average Metric: 20.00 / 30 (66.7%): 100%|██████████| 30/30 [00:00<00:00, 32.39it/s]

2025/08/08 19:55:01 INFO dspy.evaluate.evaluate: Average Metric: 20 / 30 (66.7%)


,passage,question,options,statements,example_answer,original_index,reasoning,pred_answer,validate_answer
0,문학이 사회와 그 구성원의 삶을 반영한다는 명제는 법의 영역에도 적용된다. 문학적 서사는 한 시대의 법인식과 정의관을 ...,윗글의 내용과 일치하지 않는 것은?,"①형법 개혁 운동은 범죄소설 열풍의 계기이자 성과였다. ②뉴게이트 소설은 범죄를 질병으로, 형벌을 치료로 이해한 당대 ...",,2,0,"1) 지문에서 형법 개혁 운동과 범죄소설 열풍의 관계를 명시적으로 언급하며, 개혁의 원동력이 되었다고 설명한다. 2) ...",2,✔️ [True]
1,문학이 사회와 그 구성원의 삶을 반영한다는 명제는 법의 영역에도 적용된다. 문학적 서사는 한 시대의 법인식과 정의관을 ...,㉠∼㉤에 대한 이해로 적절하지 않은 것은?,①㉠은 죄에 비해 과한 형을 구형하거나 사형 선고를 남발하는 현상을 가리킨다. ②㉡은 살아남기 위해 주어진 계급적 위치...,,2,1,"1. ㉠ ""피에 굶주린 형법전""은 과도한 형벌과 사형 선고를 비판하는 표현으로, ①의 설명과 일치한다. 2. ㉡ ""자연...",4,
2,문학이 사회와 그 구성원의 삶을 반영한다는 명제는 법의 영역에도 적용된다. 문학적 서사는 한 시대의 법인식과 정의관을 ...,윗글에서 추론한 것으로 가장 적절한 것은?,①디킨스는 법의 부조리에 대한 비판과 범죄의 해악에 대한 훈계를 한 작품에서 동시에 수행할 수 없다고 보았을 것이다. ...,,5,2,1. 지문에서 디킨스는 법의 부정의를 고발하면서도 해학과 권선징악을 통해 법질서를 교란하지 않는 방식을 선택했다고 언급...,5,✔️ [True]
3,동서양의 전설에 나오는 귀신 중 흡혈귀는 문학의 소재로 오래 활용되었다. 특히 흡혈귀는 슬라브 또는 헝가리의 전설에 자...,윗글의 내용과 일치하지 않는 것은?,①코프로포르피리노젠I은 포르피린의 한 종류이다. ②미오글로빈과 시토크롬은 헴을 보철그룹으로 가지고 있는 단백질이다. ③...,,4,3,1) 지문에서 코프로포르피리노젠I은 포르피린 합성 중간물질로 명시되어 있으므로 ①은 일치한다. 2) 미오글로빈과 시토크...,4,✔️ [True]
4,동서양의 전설에 나오는 귀신 중 흡혈귀는 문학의 소재로 오래 활용되었다. 특히 흡혈귀는 슬라브 또는 헝가리의 전설에 자...,윗글에서 추론한 내용으로 가장 적절한 것은?,①콕스는 조지 3세의 모발에서 비소 대사와 관련된 효소 유전자의 결함을 찾고자 하였을 것이다. ②조지 3세는 불면증과 ...,,2,4,"1) 콕스는 조지 3세의 모발에서 헴 합성과 연관된 유전자 결함을 찾으려 했으나 성공하지 못했고, 대신 비소를 발견했다...",2,✔️ [True]
5,동서양의 전설에 나오는 귀신 중 흡혈귀는 문학의 소재로 오래 활용되었다. 특히 흡혈귀는 슬라브 또는 헝가리의 전설에 자...,"<보기>를 바탕으로 본문을 이해할 때, 가장 적절한 것은?","①효소 ㉠, ㉡의 산물은 도스포르피린증 환자의 체내에 축적될 것이다. ②효소 ㉢의 산물이 코프로포르피리노젠I로 전환되는...",<보 기> 헴 합성은 (가)와 같은 다단계 효소 촉매 과정에 의하여 일어난다. 효소는 ‘기질’의 화학적 구조를 변화시키...,3,5,"1. **①** 도스포르피린증은 효소 ㉠ 결핍으로 발생하며, 기질인 델타아미노레불린산이 축적됩니다. 산물인 포르피린 A...",3,✔️ [True]
6,"기존의 역사가들이 민주주의, 노예제와 같은 정치․사회제도의 모델을 찾기 위해 고대 그리스와 로마에 주목했다면, 최근에는...",윗글에 대한 이해로 적절하지 않은 것은?,①플라톤은 파이데라스티아의 대상을 일정한 지적 성장 단계의 소년으로 한정했다. ②크세노폰은 에라스테스를 소년의 육체를 ...,,2,6,"1) 플라톤은 ""이성을 갖기 시작한 나이의 소년들만을 사랑한다""고 서술하여 파이데라스티아의 대상을 지적 성장 단계의 소...",2,✔️ [True]
7,"기존의 역사가들이 민주주의, 노예제와 같은 정치․사회제도의 모델을 찾기 위해 고대 그리스와 로마에 주목했다면, 최근에는...",윗글로 보아 다음 설명 중 가장 적절한 것은?,①아테네와 스파르타에서는 모두 이십 대 청년이 에로메노스에서 배제되었다. ②아테네에서와 달리 스파르타에서의 에라스테스는...,,3,7,"1. 아테네에서는 17세 이하의 소년이 에로메노스 역할을 했고, 스파르타에서는 30세까지도 이 역할을 유지했다는 점에서...",3,✔️ [True]
8,"기존의 역사가들이 민주주의, 노예제와 같은 정치․사회제도의 모델을 찾기 위해 고대 그리스와 로마에 주목했다면, 최근에는...","윗글과 <보기>를 연결하여 평가할 때, 가장 적절한 것은?",①도자기에 그려진 장면은 에로메노스와 에라스테스 관계에 대한 골든의 해석과 상충하는군. ②그리스 도자기의 소년애 장면은...,<보 기> 고대 그리스 도자기에 묘사된 소년애 장면은 이성애 장면에 비해 훨씬 덜 노골적이다. 이는 자유민 소년이 성적...,2,8,1. 도자기 장면과 골든의 해석: 도자기 장면이 덜 노골적이라는 것은 골든이 주장한 '소년의 명예와 존엄 배려'와 일치...,4,
9,사법심사는 다수주의의 예외로 간주되기도 한다. 민주적 절차로 선출된 의회나 행정부의 결정이 합헌 여부를 기준으로 무효화...,네 가지 모델에 대한 설명으로 가장 적절한 것은?,①긍정적 반응 모델은 연방대법원의 전문성과 공정성에 대한 대중의 불신을 반영한다. ②연방대법원의 결정을 여론이 즉각 수...,,3,9,"1. 긍정적 반응 모델은 연방대법원의 전문성과 공정성에 대한 대중의 신뢰를 전제로 하므로, ①번은 잘못된 설명이다. 2...",3,✔️ [True]


🏃 View run Cot_after_optimization at: http://127.0.0.1:5000/#/experiments/366534923737239846/runs/a31a35fc58834adfb9c1e5f3d7680854
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/366534923737239846


[Trace(trace_id=tr-a0cccf2c7576a778ffc568019ffe2469), Trace(trace_id=tr-99c3d240949997128e8d4e8a02f1a5aa), Trace(trace_id=tr-4c71e9d714ca684573ea218350815f34), Trace(trace_id=tr-f4f02ddf8324221840bee374b2e8a0df), Trace(trace_id=tr-095400196010a9de5dc10941c69597e6), Trace(trace_id=tr-2cbbb6bdcbbe4be9374c4359306fa0c2), Trace(trace_id=tr-5942757dfb707c6bb631af9b2cecdf09), Trace(trace_id=tr-5182fe140bcc9740f66e208b09faa55b), Trace(trace_id=tr-a4b10771492d35df2d2962d008ffbde3), Trace(trace_id=tr-f0ac31bd0006cfcb2c677939fe6c8ab5)]